# Homework 4 Notebook

### 5.2 Warming up

In [4]:
import numpy as np


def run_experiemnt(n: int) -> tuple[float, float]:
    cos_values = np.ndarray(shape=(n))
    x = np.random.normal(0.0, 1.0, n)
    cos_2 = lambda x: np.cos(x)# * np.cos(x)
    cos_values = cos_2(x)
    return cos_values.mean(), cos_values.std()

n = 1000000
mean, std = run_experiemnt(n)
print(F"Mean: {mean}")
print(F"STD: {std}")
print(F"[{mean - 1.96*(std/np.sqrt(n))};{mean + 1.96*(std/np.sqrt(n))}]")

Mean: 0.6069426250283675
STD: 0.4470797750384943
[0.606066348669292;0.607818901387443]


### 5.3 

In [3]:
x = np.abs(-1)